# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [43]:
# import libraries

import pandas as pd
import numpy as np
import re

from sqlalchemy import create_engine

import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer


from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.multioutput import MultiOutputClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import precision_recall_fscore_support
from sklearn.preprocessing import StandardScaler


import pickle

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///DisasterResponse.db')
engine.table_names()
df = pd.read_sql_table('Message', engine)
X = df.message
y =df.loc[:, 'related':'direct_report']

### 2. Write a tokenization function to process your text data

In [3]:
url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
def tokenize(text):
    
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, 'urlplaceholder')
        
    tokens = word_tokenize(text)
    lemmatizer = WordNetLemmatizer()
        
    clean_tokens = []
    for tok in tokens:
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)
    
    return clean_tokens

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [4]:
pipeline = pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidt', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier())),
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [5]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

In [6]:
y_test.shape

(5244, 36)

In [9]:
def get_results(y_test, y_pred):
    results = []
    categories = y_test.columns.tolist()

    for cat, col in zip(categories, range(len(y_pred))):
        precision, recall, f_score, support = precision_recall_fscore_support(y_test[cat], y_pred[:,col], average='weighted')
        results.append({'Category': cat,
                        'precision': precision,
                        'recall': recall,
                        'f_score': f_score})

    results = pd.DataFrame(results, columns=['Category', 'precision', 'recall', 'f_score'])
    print('Average f_score:', results['f_score'].mean())
    print('Average precision:', results['precision'].mean())
    print('Average recall:', results['recall'].mean())

    return results


In [10]:
results = get_results(y_test, y_pred)

Average f_score: 0.927722531653
Average precision: 0.931309674438
Average recall: 0.942007797271


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [ ]:
from sklearn.ensemble import GradientBoostingRegressor

num_estimators = [500,1000]
learn_rates = [0.02, 0.05]
max_depths = [1, 2]
min_samples_leaf = [5,10]
min_samples_split = [5,10]

# assign the parameters to the named step in the pipeline
param_grid = {'clf__num_estimators': num_estimators,
              'clf__learning_rate': learn_rates,
              'max_depth': max_depths,
              'min_samples_leaf': min_samples_leaf,
              'min_samples_split': min_samples_split}

grid_search = GridSearchCV(GradientBoostingRegressor(loss='huber'),
                           param_grid, cv=3, return_train_score=True)
grid_search.fit(x_train, y_train)

### 6. Improve your model
Use grid search to find better parameters. 

In [14]:
sorted(pipeline.get_params().keys())

['clf',
 'clf__estimator',
 'clf__estimator__bootstrap',
 'clf__estimator__class_weight',
 'clf__estimator__criterion',
 'clf__estimator__max_depth',
 'clf__estimator__max_features',
 'clf__estimator__max_leaf_nodes',
 'clf__estimator__min_impurity_decrease',
 'clf__estimator__min_impurity_split',
 'clf__estimator__min_samples_leaf',
 'clf__estimator__min_samples_split',
 'clf__estimator__min_weight_fraction_leaf',
 'clf__estimator__n_estimators',
 'clf__estimator__n_jobs',
 'clf__estimator__oob_score',
 'clf__estimator__random_state',
 'clf__estimator__verbose',
 'clf__estimator__warm_start',
 'clf__n_jobs',
 'memory',
 'steps',
 'tfidt',
 'tfidt__norm',
 'tfidt__smooth_idf',
 'tfidt__sublinear_tf',
 'tfidt__use_idf',
 'vect',
 'vect__analyzer',
 'vect__binary',
 'vect__decode_error',
 'vect__dtype',
 'vect__encoding',
 'vect__input',
 'vect__lowercase',
 'vect__max_df',
 'vect__max_features',
 'vect__min_df',
 'vect__ngram_range',
 'vect__preprocessor',
 'vect__stop_words',
 'vect__s

In [17]:
max_depths = [1, 2]
min_samples_leaf = [5,10]
min_samples_split = [5,10]

parameters = {
    'clf__estimator__max_depth': max_depths,
    'clf__estimator__min_samples_leaf': min_samples_leaf,
    'clf__estimator__min_samples_split': min_samples_split
}

cv = GridSearchCV(pipeline, param_grid = parameters)


### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [18]:
cv.fit(X_train, y_train)
y_pred = cv.predict(X_test)

In [21]:
results2 = get_results(y_test, y_pred)
results2

Average f_score: 0.894186130454
Average precision: 0.878088668635
Average recall: 0.926958852445


/opt/conda/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


,Category,precision,recall,f_score
0,related,0.563931,0.750953,0.644142
1,request,0.687786,0.829329,0.751955
2,offer,0.990108,0.995042,0.992569
3,aid_related,0.758901,0.594012,0.443125
4,medical_help,0.845530,0.919527,0.880977
5,medical_products,0.899676,0.948513,0.923449
6,search_and_rescue,0.952150,0.975782,0.963821
7,security,0.966719,0.983219,0.974899
8,military,0.941758,0.970442,0.955885
9,child_alone,1.000000,1.000000,1.000000


### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [44]:
moc = MultiOutputClassifier(DecisionTreeClassifier())
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidt', TfidfTransformer()),
        ('clf', moc)
    ])



In [45]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
pipeline.fit(X_train, y_train)
y_pred = pipeline.predict(X_test)
results3 = get_results(y_test, y_pred)
results3

Average f_score: 0.930206602891
Average precision: 0.929243902826
Average recall: 0.931291846767


,Category,precision,recall,f_score
0,related,0.736017,0.744661,0.739821
1,request,0.845217,0.846301,0.845749
2,offer,0.990095,0.992372,0.991232
3,aid_related,0.689856,0.689741,0.689798
4,medical_help,0.894969,0.895309,0.895138
5,medical_products,0.936791,0.940694,0.938623
6,search_and_rescue,0.961656,0.962243,0.961948
7,security,0.968449,0.969680,0.969062
8,military,0.962946,0.962243,0.962590
9,child_alone,1.000000,1.000000,1.000000


### 9. Export your model as a pickle file

In [47]:
pickle_out = open('model_ml.pkl', 'wb')
pickle.dump(cv, pickle_out)
pickle_out.close()

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.